## Pérdida exponencial y AdaBoost

Si $\ell(\tilde{y},\hat{y})=\exp(-\tilde{y}\hat{y})$, $\tilde{y}\in\{-1,+1\}$, el objetivo FSAM en la iteración $m$ es:
$$\begin{align*}
L_m(\beta,\boldsymbol{\theta})%
&=\sum_{i=1}^N \exp(-\tilde{y}_i(f_{m-1}(\boldsymbol{x}_i)+\beta F(\boldsymbol{x}_i;\boldsymbol{\theta})))\\%
&=\sum_{i=1}^N w_{im}\exp(-\beta\tilde{y}_i F(\boldsymbol{x}_i;\boldsymbol{\theta}))%
\qquad\text{con}\qquad w_{im}=\exp(-\tilde{y}_i f_{m-1}(\boldsymbol{x}_i))\\%
&=e^{\beta}\sum_{\tilde{y}_i\neq F(\boldsymbol{x}_i;\boldsymbol{\theta})} w_{im}
+e^{-\beta}\sum_{\tilde{y}_i=F(\boldsymbol{x}_i;\boldsymbol{\theta})} w_{im}\\%
&=e^{\beta}\sum_{\tilde{y}_i\neq F(\boldsymbol{x}_i;\boldsymbol{\theta})} w_{im}
+e^{-\beta}\left(\sum_{i=1}^N w_{im}-\sum_{\tilde{y}_i\neq F(\boldsymbol{x}_i;\boldsymbol{\theta})} w_{im}\right)\\%
&=(e^{\beta}-e^{-\beta})\sum_{i=1}^N w_{im}\mathbb{I}(\tilde{y}_i\neq F(\boldsymbol{x}_i;\boldsymbol{\theta}))%
+e^{-\beta}\sum_{i=1}^N w_{im}
\end{align*}$$
La minimización de $L_m$ puede hacerse en dos pasos. Primero hallamos $\boldsymbol{\theta}_m$ a partir de los datos ponderados:
$$\boldsymbol{\theta}_m=\operatorname*{argmin}_{\boldsymbol{\theta}}%
\sum_{i=1}^N w_{im}\mathbb{I}(\tilde{y}_i\neq F(\boldsymbol{x}_i;\boldsymbol{\theta}))$$
y luego $\beta_m$ mediante minimización en $\beta$ de $L_m(\beta,\boldsymbol{\theta}_m)$:
$$\beta_m=\operatorname*{argmin}_{\beta}\;L_m(\beta,\boldsymbol{\theta}_m)%
=\frac{1}{2}\log\frac{1-\operatorname{err}_m}{\operatorname{err}_m}%
\qquad\text{con}\qquad\operatorname{err}_m=\frac{1}{\sum_{i=1}^N w_{im}}%
\sum_{i=1}^N w_{im}\mathbb{I}(\tilde{y}_i\neq F_m(\boldsymbol{x}_i))$$
**Adaboost** halla $F_m(\cdot)$ y $\beta_m$ en la iteración $m$. Los pesos de los datos para la primera iteración son $w_{i1}=1/N$. Una vez hallados $F_m(\cdot)$ y $\beta_m$ en la iteración $m$, los pesos de los datos para la iteración $m+1$ se calculan fácilmente:
$$\begin{align*}
w_{i,m+1}&=\exp(-\tilde{y}_i f_m(\boldsymbol{x}_i))\\%
&=\exp(-\tilde{y}_i f_{m-1}(\boldsymbol{x}_i)-\tilde{y}_i\beta_m F_m(\boldsymbol{x}_i))\\%
&=w_{im}\exp(-\tilde{y}_i\beta_m F_m(\boldsymbol{x}_i))\\%
&=w_{im}\exp(\beta_m(2\mathbb{I}(\tilde{y}_i\neq F_m(\boldsymbol{x}_i))-1))\\%
&=w_{im}\exp(2\beta_m\mathbb{I}(\tilde{y}_i\neq F_m(\boldsymbol{x}_i)))\exp(-\beta_m)%
\end{align*}$$
El factor $\exp(-\beta_m)$ se puede ignorar ya que es constante para todos los datos en el objetivo FSAM de la iteración $m+1$. Así pues, los pesos de los datos para la iteración $m+1$ son:
$$w_{i,m+1}=\begin{cases}
w_{im}\exp(2\beta_m) & \text{si $\tilde{y}_i\neq F_m(\boldsymbol{x}_i)$}\\%
w_{im}               & \text{en otro caso}%
\end{cases}$$
El modelo ajustado para clasificación binaria es $f(\boldsymbol{x})=\operatorname{sgn}(\sum_m\beta_m F_m(\boldsymbol{x}))$; para regresión y clasificación multi-clase se usan variantes convenientemente adaptadas. En general, AdaBoost es muy sensible a outliers ya que los pesos de los datos mal clasificados crecen exponencialmente.

Otro aspecto que limita la aplicabilidad de AdaBoost es que no facilita la estimación probabilidades. En teoría, el riesgo de un modelo $f(\boldsymbol{x})$ con pérdida exponencial es:
$$\mathbb{E}[\exp(-\tilde{y}f(\boldsymbol{x}))\mid\boldsymbol{x}]%
=p(\tilde{y}=1\mid\boldsymbol{x})\exp(-f(\boldsymbol{x}))%
+p(\tilde{y}=-1\mid\boldsymbol{x})\exp(f(\boldsymbol{x}))$$
Derivando el riesgo con respecto a $f(\boldsymbol{x})$ e igualando a cero, obtenemos que el modelo de mínimo riesgo teórico halla la mitad de la log-odds:
$$f(\boldsymbol{x})=\frac{1}{2}\log\frac{p(\tilde{y}=1\mid\boldsymbol{x})}{p(\tilde{y}=-1\mid\boldsymbol{x})}$$
Nótese que este resultado justifica la aplicación del operador signo al modelo ajustado para clasificación binaria.

**Ejemplo:**  clasificación de correos en spam y no-spam

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv("https://github.com/empathy87/The-Elements-of-Statistical-Learning-Python-Notebooks/blob/master/data/Spam.txt?raw=True")
is_test = df.test.values; y = df.spam.values; X = df.drop(['test','spam'], axis=1).to_numpy(copy=True)
X_train, X_test = X[is_test == 0], X[is_test == 1]
y_train, y_test = y[is_test == 0], y[is_test == 1]
ntrees_list = [10, 50, 100, 200, 300, 400, 500]
for ntrees in ntrees_list:
    clf = AdaBoostClassifier(n_estimators=ntrees, random_state=10, learning_rate=0.2).fit(X_train, y_train)
    y_test_hat = clf.predict(X_test)
    acc = accuracy_score(y_test, y_test_hat)
    print(f'AdaBoosting {ntrees} trees, test err {1 - acc:.1%}')

AdaBoosting 10 trees, test err 10.9%
AdaBoosting 50 trees, test err 7.2%
AdaBoosting 100 trees, test err 5.9%
AdaBoosting 200 trees, test err 5.7%
AdaBoosting 300 trees, test err 5.7%
AdaBoosting 400 trees, test err 5.5%
AdaBoosting 500 trees, test err 5.5%
